<a href="https://colab.research.google.com/github/fabriziobasso/kaggle/blob/main/Bloomberg_Data_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install blpapi --index-url=https://bcms.bloomberg.com/pip/simple/

Looking in indexes: https://bcms.bloomberg.com/pip/simple/


In [6]:
!pip install xbbg
!pip install pdblp

  Preparing metadata (setup.py) ... done
  Created wheel for pdblp: filename=pdblp-0.1.8-py3-none-any.whl size=18646 sha256=6570a531ea5782f555ec6d245417b18942912c65d43b63d8fc157c92922df0b1
  Stored in directory: /root/.cache/pip/wheels/d4/5d/a3/c5bf685e879e8e4899c73d59048926f1846a08e5d141adcbda
Successfully built pdblp


In [3]:
import pandas as pd
import numpy as np

from xbbg import blp, pipeline

# Reference Data `BDP` and `BDS`

In [4]:
blp.__version__

'0.7.7'

In [5]:
blp.bdp('AAPL US Equity', flds=['Security_Name', 'Last_Price'])

ConnectionError: Cannot connect to Bloomberg

In [ ]:
blp.bdp('6758 JP Equity', flds='Crncy_Adj_Mkt_Cap', Eqy_Fund_Crncy='USD')

In [ ]:
holders = blp.bds('AMZN US Equity', flds='All_Holders_Public_Filings', cache=True)
(
    holders
    .loc[:, ~holders.columns.str.contains(
        f'holder_id|portfolio_name|change|number|'
        f'metro|percent_of_portfolio|source'
    )]
    .rename(
        index=lambda tkr: tkr.replace(' Equity', ''),
        columns={
            'holder_name_': 'holder',
            'position_': 'position',
            'filing_date__': 'filing_dt',
            'percent_outstanding': 'pct_out',
            'insider_status_': 'insider',
        }
    )
).head()

In [ ]:
blp.dividend('SPY US Equity', start_date='2019')

In [ ]:
blp.earning('FB US Equity', Eqy_Fund_Year=2018, Number_Of_Periods=2)

# Historical Data

Historical data with Excel compatible overrides

In [ ]:
blp.bdh(
    tickers='SHCOMP Index', flds=['high', 'low', 'last_price'],
    start_date='2019-11', end_date='2020', Per='W', Fill='P', Days='A',
)

Dividend / split adjustments

In [ ]:
pd.concat([
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=True, CshAdjAbnormal=True, CapChg=True
    ).rename(columns={'Px_Last': 'Px_Adj'}),
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=False, CshAdjAbnormal=False, CapChg=False
    ).rename(columns={'Px_Last': 'Px_Raw'}),
], axis=1)

# Intraday Bars

In [ ]:
cur_dt = pd.Timestamp('today', tz='America/New_York').date()
recent = pd.bdate_range(end=cur_dt, periods=2, tz='America/New_York')
pre_dt = max(filter(lambda dd: dd < cur_dt, recent))
pre_dt.date()

In [ ]:
blp.bdib('QQQ US Equity', dt=pre_dt, session='day').tail()

In [ ]:
blp.bdib('388 HK Equity', dt=pre_dt, session='am_open_7')

# Intraday Tick Data

In [ ]:
blp.bdtick('QQQ US Equity', dt=pre_dt).tail(10)

# Equity Screen `BEQS`

In [ ]:
blp.beqs('Core Capital Ratios', typ='GLOBAL').iloc[:5, :6]

# Subscription

`blp.live` will yield market data as `dict`

In [ ]:
async for snap in blp.live(['ESA Index', 'NQA Index'], max_cnt=2):
    print(snap)

# Pipelines

In [ ]:
cur_dt = pd.Timestamp('today', tz='America/New_York').date()
recent = pd.bdate_range(end=cur_dt, periods=2, tz='America/New_York')
pre_dt = max(filter(lambda dd: dd < cur_dt, recent))

fx = blp.bdib('JPY Curncy', dt=pre_dt)
jp = pd.concat([
    blp.bdib(ticker, dt=pre_dt, session='day')
    for ticker in ['7974 JP Equity', '9984 JP Equity']
], axis=1)
jp.tail()

Get `close` prices and convert to USD

In [ ]:
prices = (
    jp
    .pipe(pipeline.get_series, col='close')
    .pipe(pipeline.apply_fx, fx=fx)
    .tz_convert('Asia/Tokyo')
)
prices.tail()

## Customized Pipelines

VWAP for intraday bar data

In [ ]:
def vwap(data: pd.DataFrame, fx=None, name=None) -> pd.Series:
    return pd.Series({
        ticker: (
            data[ticker][['close', 'volume']].prod(axis=1).sum()
            if fx is None else (
                data[ticker].close
                .pipe(pipeline.apply_fx, fx)
                .close
                .mul(data[ticker].volume)
                .sum()
            )
        ) / data[ticker].volume.sum()
        for ticker in data.columns.get_level_values(0).unique()
    }, name=name)

VWAP in local currency

In [ ]:
jp.pipe(vwap, name=jp.index[-1].date())

VWAP in USD

In [ ]:
jp.pipe(vwap, fx=fx, name=jp.index[-1].date())

Total traded volume as of time in day for past few days

In [ ]:
jp_hist = pd.concat([
    pd.concat([
        blp.bdib(ticker, dt=dt, session='day')
        for ticker in ['7974 JP Equity', '9984 JP Equity']
    ], axis=1)
    for dt in pd.bdate_range(end='today', periods=10)[:-1]
], sort=False)
unique(jp_hist.index.date)

In [ ]:
def drop_zeros(data: pd.DataFrame) -> pd.DataFrame:
    return (
        data
        .replace(0, np.nan)
        .dropna(how='all')
        .replace(np.nan, 0)
    )

In [ ]:
def traded_volume(data: pd.DataFrame, asof: str) -> pd.DataFrame:
    return (
        data
        .pipe(pipeline.get_series, col='volume')
        .between_time('0:00', asof)
        .resample('B')
        .sum()
        .pipe(drop_zeros)
    )

In [ ]:
jp_hist.pipe(traded_volume, asof='10:00')

In [ ]:
jp_hist.pipe(traded_volume, asof='11:00')